In [1]:
pip install analytical_option_formulae

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement analytical_option_formulae (from versions: none)
ERROR: No matching distribution found for analytical_option_formulae


In [3]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq
import matplotlib.pylab as plt
from scipy.interpolate import interp1d
from scipy.optimize import least_squares
import matplotlib.ticker as ticker
from scipy.integrate import quad

import sys

sys.path.append("..")

from analytical_option_formulae.option_types.vanilla_option import VanillaOption

vanilla_option = VanillaOption()

# please adjust this before running, its either SPX or SPY
filename = "SPY_options"
part3_date = pd.to_datetime("20210115", format="%Y%m%d")

ModuleNotFoundError: No module named 'analytical_option_formulae'

In [ ]:
# implied volatility reporting


def implied_volatility_bs(
    S: float, K: float, r: float, price: float, T: float, options_type: str
) -> float:
    try:
        bs_model = lambda x: vanilla_option.black_scholes_model(S, K, r, x, T)
        if options_type.lower() == "call":
            implied_vol = brentq(
                lambda x: price - bs_model(x).calculate_call_price(), 1e-12, 10.0
            )
        elif options_type.lower() == "put":
            implied_vol = brentq(
                lambda x: price - bs_model(x).calculate_put_price(), 1e-12, 10.0
            )
        else:
            raise NameError("Payoff type not recognized")
    except Exception:
        implied_vol = np.nan

    return implied_vol

In [6]:
df = pd.read_csv(f"../data/{filename}.csv")
df["mid"] = 0.5 * (df["best_bid"] + df["best_offer"])
df["strike"] = df["strike_price"] * 0.001
df["payoff"] = df["cp_flag"].map(lambda x: "call" if x == "C" else "put")
df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")
df["exdate"] = pd.to_datetime(df["exdate"], format="%Y%m%d")
df["days_to_expiry"] = (df["exdate"] - df["date"]).dt.days
df["years_to_expiry"] = df["days_to_expiry"] / 365
df = df[df["exdate"] == part3_date]
df = df.reset_index(drop=True)

# setup rates calculator
rates_df = pd.read_csv("zero_rates_20201201.csv")
rate_interpolate = interp1d(rates_df["days"], rates_df["rate"])
df["rates"] = (
    rate_interpolate(df["days_to_expiry"]) / 100.0
)  # make it in fractions so i dont forget

try:
    if filename.lower() == "spy_options":
        S = 366.02
    elif filename.lower() == "spx_options":
        S = 3662.45
    else:
        raise NameError("unknown input file")
except Exception as e:
    print(e)

# impl market volatility column
df["vols"] = df.apply(
    lambda x: implied_volatility_bs(
        S, x["strike"], x["rates"], x["mid"], x["years_to_expiry"], x["payoff"]
    ),
    axis=1,
)
axis = (1,)
df.dropna(inplace=True)

NameError: name 'filename' is not defined

In [5]:
# create market DF for each timestamp
call_df = df[df["payoff"] == "call"]
put_df = df[df["payoff"] == "put"]
strikes = sorted(df["strike"].unique())
impliedvols = []
impliedvols_bach = []
option_type = []
option_price = []
for K in strikes:
    if K > S:
        impliedvols.append(call_df[call_df["strike"] == K]["vols"].values[0])
        option_type.append("call")
        option_price.append(call_df[call_df["strike"] == K]["mid"].values[0])
    else:
        impliedvols.append(put_df[put_df["strike"] == K]["vols"].values[0])
        option_type.append("put")
        option_price.append(put_df[put_df["strike"] == K]["mid"].values[0])

day_market_df = pd.DataFrame(
    {
        "strike": strikes,
        "vol": impliedvols,
        "option_type": option_type,
        "option_price": option_price,
    }
)
day_market_df

NameError: name 'df' is not defined

In [82]:
start_date = pd.to_datetime("2020-12-01")
end_date = pd.to_datetime("2021-01-15")
day_diff = (end_date - start_date).days
T = day_diff / 365.0
rate = rate_interpolate(day_diff) / 100.0

# use interpolate to get sigma
bs_sigma = np.interp(S, day_market_df["strike"], day_market_df["vol"])
print(f"BS sigma using interpolation : {bs_sigma}")

# get call price to verif
bs_call_price_with_sigma_interp = vanilla_option.black_scholes_model(
    S, S, rate, bs_sigma, T
).calculate_call_price()
print(f"BS Call using sigma interp : {bs_call_price_with_sigma_interp}")

BS sigma using interpolation : 0.1849096526276905
BS Call using sigma interp : 95.2990207952223


In [83]:
# find price of ATM option
opt_price_interp = np.interp(S, day_market_df["strike"], day_market_df["option_price"])


# bachelier sigma from interpolated price of the ATM option
def bach_sigma(v_call, rate, T, S):
    return (v_call * np.exp(rate * T)) / (S * np.sqrt(T / (2 * np.pi)))


bach_sigma = bach_sigma(opt_price_interp, rate, T, S)



bach_call_price = vanilla_option.bachelier_model(
    S, S, rate, bach_sigma, T
).calculate_call_price()


print(f"Bach Call using sigma interp : {bach_call_price} vs {opt_price_interp}")

Bach Call using sigma interp : 93.58200000000014 vs 93.58200000000012


# Exotic No.1 : Black-Scholes

Expected Black Scholes payoff is defined as

$$
E[V_T]= {S_0}^\frac{1}{3}e^{\frac{rT}{3}}e^{\frac{-\sigma^2T}{9}} + 1.5(log{S_0} + (r-\frac{\sigma^2}{2})T) + 10
$$

Therefore, the price is

$$
V_0 = e^{-rT}E[V_T]
$$


In [84]:
# black scholes payoff
def bs_price(S, rate, sigma, T):
    return np.exp(-rate * T) * (
        (
            np.power(S, 1.0 / 3.0)
            * np.exp((rate - 0.5 * sigma**2) * T * (1.0 / 3.0))
            * np.exp(0.5 * (1.0 / 9.0) * T * sigma**2)
            + 1.5 * (np.log(S) + (rate - 0.5 * sigma**2) * T)
            + 10
        )
    )

# Exotic No.1 : Bachelier

Expected Bachelier payoff defined as

$$
E[V_T] = \frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} (S_0 + \sigma S_0 \sqrt{T} x)^\frac{1}{3} e^\frac{-x^2}{2}\,dx +
\frac{1}{\sqrt{2\pi}}\int_{-\infty}^{\infty} 1.5log(S_0 + \sigma S_0 \sqrt{T} x) e^\frac{-x^2}{2}\,dx
+10
$$

Note $$S_T = S_0 + \sigma S_0W_T$$

Therefore, the price is

$$
V_0 = e^{-rT}E[V_T]
$$


In [85]:
def integrand_1(x, S, sigma, T):
    return (
        (1 / np.sqrt(2 * np.pi))
        * np.power((S + sigma * S * np.sqrt(T) * x), 1.0 / 3.0)
        * np.exp(-0.5 * np.power(x, 2))
    )


def integrand_2(x, S, sigma, T):
    return (
        (1 / np.sqrt(2 * np.pi))
        * 1.5
        * np.log(S + sigma * S * np.sqrt(T) * x)
        * np.exp(-0.5 * np.power(x, 2))
    )


def bachelier_price(S, rate, sigma, T):
    lower_bound = -1 / (sigma * np.sqrt(T))  # log term lower bound
    I_1 = quad(lambda x: integrand_1(x, S, sigma, T), lower_bound, np.inf)
    I_2 = quad(lambda x: integrand_2(x, S, sigma, T), lower_bound, np.inf)
    V_0_bachelier = np.exp(-rate * T) * (I_1[0] + I_2[0] + 10)
    return V_0_bachelier

# Exotic No.1 : SABR & Static Replication

For SABR payoff, we must retrieve the volatility using previous calibration result

$$
h(S_T) = S_T^{\frac{1}{3}} + 1.5 log(S_T) + 10 \\
h''(S_T) = -\frac{2}{9}S_T^{-\frac{5}{3}} - 1.5\frac{1}{S_T^2} \\
F = S_0 e ^ {rT}
$$

Therefore, the price is

$$
V_0 = e^{-rT}h(F) + \int_{0}^{F} h''(K)P(K) \,dK + \int_{F}^{\infty} h''(K)C(K) \,dK
$$


In [86]:
# SABR Related
# change SABR params


spy_sabr_params = {
    45: {
        "alpha": 0.9081326337814014,
        "rho": -0.4887794457550238,
        "nu": 2.7285163417661487,
    },
}


spx_sabr_params = {
    45: {
        "alpha": 1.8165044370781172,
        "rho": -0.4043017672449347,
        "nu": 2.790158312103804,
    },
}



try:
    if filename.lower() == "spy_options":
        alpha = spy_sabr_params[day_diff]["alpha"]

        rho = spy_sabr_params[day_diff]["rho"]

        nu = spy_sabr_params[day_diff]["nu"]


        beta = 0.7

    elif filename.lower() == "spx_options":
        alpha = spx_sabr_params[day_diff]["alpha"]

        rho = spx_sabr_params[day_diff]["rho"]

        nu = spx_sabr_params[day_diff]["nu"]


        beta = 0.7
    else:

        raise NameError("unknown input file")


except Exception as e:
    print(e)



# i just use prof Tee



def SABR(
    F: float, K: float, T: float, alpha: float, beta: float, rho: float, nu: float
) -> float:
    X = K

    # if K is at-the-money-forward

    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta) ** 2) / 24) * alpha * alpha / (F ** (2 - 2 * beta))

        numer2 = 0.25 * rho * beta * nu * alpha / (F ** (1 - beta))

        numer3 = ((2 - 3 * rho * rho) / 24) * nu * nu

        VolAtm = alpha * (1 + (numer1 + numer2 + numer3) * T) / (F ** (1 - beta))

        sabrsigma = VolAtm
    else:

        z = (nu / alpha) * ((F * X) ** (0.5 * (1 - beta))) * np.log(F / X)

        zhi = np.log((((1 - 2 * rho * z + z * z) ** 0.5) + z - rho) / (1 - rho))

        numer1 = (((1 - beta) ** 2) / 24) * ((alpha * alpha) / ((F * X) ** (1 - beta)))

        numer2 = 0.25 * rho * beta * nu * alpha / ((F * X) ** ((1 - beta) / 2))

        numer3 = ((2 - 3 * rho * rho) / 24) * nu * nu

        numer = alpha * (1 + (numer1 + numer2 + numer3) * T) * z

        denom1 = ((1 - beta) ** 2 / 24) * (np.log(F / X)) ** 2

        denom2 = (((1 - beta) ** 4) / 1920) * ((np.log(F / X)) ** 4)

        denom = ((F * X) ** ((1 - beta) / 2)) * (1 + denom1 + denom2) * zhi

        sabrsigma = numer / denom


    return sabrsigma

In [87]:
def h_func(F):
    return np.power(F, 1.0 / 3.0) + 1.5 * np.log(F) + 10


def h_second_deriv(F):
    return (-2 / 9) * np.power(F, -5.0 / 3.0) - 1.5 * np.power(F, -2)


def sabr_put(x, S, rate, T, alpha, beta, rho, nu):
    sigma_p = SABR(S * np.exp(rate * T), x, T, alpha, beta, rho, nu)
    bsmodel = vanilla_option.black_scholes_model(S, x, rate, sigma_p, T)
    return bsmodel.calculate_put_price()


def sabr_call(x, S, rate, T, alpha, beta, rho, nu):
    sigma_c = SABR(S * np.exp(rate * T), x, T, alpha, beta, rho, nu)
    bsmodel = vanilla_option.black_scholes_model(S, x, rate, sigma_c, T)
    return bsmodel.calculate_call_price()


def sabr_price(S, rate, T):
    F = S * np.exp(rate * T)
    I_put = quad(
        lambda x: h_second_deriv(x) * sabr_put(x, S, rate, T, alpha, beta, rho, nu),
        0,
        F,
    )
    I_call = quad(
        lambda x: h_second_deriv(x) * sabr_call(x, S, rate, T, alpha, beta, rho, nu),
        F,
        np.inf,
    )
    V_0_SABR = np.exp(-rate * T) * h_func(F) + I_put[0] + I_call[0]
    return V_0_SABR

In [88]:
V_0_black_scholes = bs_price(S, rate, bs_sigma, T)
V_0_bachelier = bachelier_price(S, rate, bach_sigma, T)
V_0_SABR = sabr_price(S, rate, T)

# Model Free Integrated Variance - Black Scholes

$$
E\Biggl[\int_{0}^{T} \sigma_t^2 \,dt\Biggr] = 2e^{rT}\Biggl(\int_{0}^{F} \frac{P(K)}{K^2} \,dK + \int_{F}^{\infty} \frac{C(K)}{K^2} \,dK\Biggr)
$$


In [89]:
def bs_put_integ(x, S, rate, sigma, T):
    bsmodel = vanilla_option.black_scholes_model(S, x, rate, sigma, T)
    return bsmodel.calculate_put_price() / (np.power(x, 2))


def bs_call_integ(x, S, rate, sigma, T):
    bsmodel = vanilla_option.black_scholes_model(S, x, rate, sigma, T)
    return bsmodel.calculate_call_price() / (np.power(x, 2))

In [90]:
F = S * np.exp(rate * T)
I_put = quad(lambda x: bs_put_integ(x, S, rate, bs_sigma, T), 0.0, F)
I_call = quad(lambda x: bs_call_integ(x, S, rate, bs_sigma, T), F, np.inf)
bs_E_var = 2 * np.exp(rate * T) * (I_put[0] + I_call[0])
bs_E_var

0.004215400228959317

# Model Free Integrated Variance - Bachelier


In [91]:
def bach_put_integ(x, S, rate, sigma, T):
    bachmodel = vanilla_option.bachelier_model(S, x, rate, sigma, T)
    return bachmodel.calculate_put_price() / (np.power(x, 2))


def bach_call_integ(x, S, rate, sigma, T):
    bachmodel = vanilla_option.bachelier_model(S, x, rate, sigma, T)
    return bachmodel.calculate_call_price() / (np.power(x, 2))

In [92]:
F = S * np.exp(rate * T)
I_put = quad(lambda x: bach_put_integ(x, S, rate, bach_sigma, T), 0.0, F)
I_call = quad(lambda x: bach_call_integ(x, S, rate, bach_sigma, T), F, np.inf)
bach_E_var = 2 * np.exp(rate * T) * (I_put[0] + I_call[0])
bach_E_var

0.004130001006070286

# Model Free Integrated Variance - SABR & Static Replication


In [93]:
def sabr_put_integ(x, S, rate, T, alpha, beta, rho, nu):
    F = S * np.exp(rate * T)
    sigma_p = SABR(F, x, T, alpha, beta, rho, nu)
    bsmodel = vanilla_option.black_scholes_model(S, x, rate, sigma_p, T)
    return bsmodel.calculate_put_price() / (np.power(x, 2))


def sabr_call_integ(x, S, rate, T, alpha, beta, rho, nu):
    F = S * np.exp(rate * T)
    sigma_c = SABR(F, x, T, alpha, beta, rho, nu)
    bsmodel = vanilla_option.black_scholes_model(S, x, rate, sigma_c, T)
    return bsmodel.calculate_call_price() / (np.power(x, 2))

In [94]:
F = S * np.exp(rate * T)
I_put = quad(lambda x: sabr_put_integ(x, S, rate, T, alpha, beta, rho, nu), 0.0, F)
I_call = quad(lambda x: sabr_call_integ(x, S, rate, T, alpha, beta, rho, nu), F, np.inf)
sabr_E_var = 2 * np.exp(rate * T) * (I_put[0] + I_call[0])
sabr_E_var

0.00635098900700971

In [95]:
out1 = {
    "bs_sigma": bs_sigma,
    "bach_sigma": bach_sigma,
    "bs_price": V_0_black_scholes,
    "bach_price": V_0_bachelier,
    "sabr_price": V_0_SABR,
}

out2 = {
    "bs_sigma": bs_sigma,
    "bach_sigma": bach_sigma,
    "bs_price": bs_E_var,
    "bach_price": bach_E_var,
    "sabr_price": sabr_E_var,
}


contract1 = pd.DataFrame.from_dict(out1.items())
contract1.columns = ["params", "value"]
contract1.to_csv(f"{filename}_contract_1.csv")
contract2 = pd.DataFrame.from_dict(out2.items())
contract2.columns = ["params", "value"]
contract2.to_csv(f"{filename}_contract_2.csv")